In [ ]:
# Import necessary libraries
import pandas as pd
import re
import unicodedata
from google.colab import files

# Function to clean text by removing punctuation, converting to lowercase, and removing diacritics
def clean_text_for_matching(text):
    """
    Cleans the input text for matching by removing punctuation, converting to lowercase,
    and removing diacritics (accents).
    """
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation using regex
    text = text.lower()  # Convert to lowercase
    # Remove diacritics
    text = ''.join(
        char for char in unicodedata.normalize('NFD', text)
        if unicodedata.category(char) != 'Mn'
    )
    return text

# Upload the lexicon and test corpus files
print("Please upload your lexicon Excel file (e.g., 'corrected_lexicon.xlsx'):")
uploaded_lexicon = files.upload()

print("Please upload your testing corpus Excel file (e.g., 'test_corpus.xlsx'):")
uploaded_test_corpus = files.upload()

# Load the uploaded lexicon file
lexicon_file = list(uploaded_lexicon.keys())[0]
df = pd.read_excel(lexicon_file)

# Adjust column names for consistency and include 'sepedi'
df.columns = ['ciluba', 'french', 'english', 'afrikaans', 'zulu', 'sepedi', 'score', 'sentiment', 'nature']

# Define language mapping for consistency, adding 'sepedi'
language_mapping = {
    'francais': 'french',
    'afrikaans': 'afrikaans',
    'zulu': 'zulu',
    'ciluba': 'ciluba',
    'english': 'english',
    'sepedi': 'sepedi'
}

# Function to preprocess the lexicon and calculate average sentiment scores for each word/phrase
def create_multi_word_translation_dict_with_sentiment(df):
    """
    Creates a translation dictionary that includes both single words and multi-word entries
    along with their average sentiment scores.
    """
    # Group by each language and English to calculate the average sentiment score
    sentiment_averages = {}
    for lang in ['french', 'afrikaans', 'zulu', 'ciluba', 'sepedi', 'english']:
        sentiment_averages[lang] = df.groupby(lang)['score'].mean().to_dict()

    translation_dicts = {
        'french': {'english': dict(zip(df['french'].str.lower(), df['english'].str.lower()))},
        'afrikaans': {'english': dict(zip(df['afrikaans'].str.lower(), df['english'].str.lower()))},
        'zulu': {'english': dict(zip(df['zulu'].str.lower(), df['english'].str.lower()))},
        'ciluba': {'english': dict(zip(df['ciluba'].str.lower(), df['english'].str.lower()))},
        'sepedi': {'english': dict(zip(df['sepedi'].str.lower(), df['english'].str.lower()))},
        'english': {
            'french': dict(zip(df['english'].str.lower(), df['french'].str.lower())),
            'afrikaans': dict(zip(df['english'].str.lower(), df['afrikaans'].str.lower())),
            'zulu': dict(zip(df['english'].str.lower(), df['zulu'].str.lower())),
            'ciluba': dict(zip(df['english'].str.lower(), df['ciluba'].str.lower())),
            'sepedi': dict(zip(df['english'].str.lower(), df['sepedi'].str.lower()))
        }
    }

    # Add multi-word entries with their sentiment averages
    for lang in ['french', 'afrikaans', 'zulu', 'ciluba', 'sepedi']:
        multi_word_entries = df[df[lang].str.contains(' ') & df['english'].str.contains(' ', na=False)]
        for _, row in multi_word_entries.iterrows():
            source_phrase = row[lang].lower()
            english_phrase = row['english'].lower()
            translation_dicts[lang]['english'][source_phrase] = english_phrase
            translation_dicts['english'][lang][english_phrase] = source_phrase

    return translation_dicts, sentiment_averages

# Create the translation dictionaries with sentiment support
translation_dicts, sentiment_averages = create_multi_word_translation_dict_with_sentiment(df)

# Function to prioritize multi-word phrase matching and calculate sentiment
def translate_and_analyze_sentiment(text, source_lang, target_lang, translation_dicts, sentiment_averages):
    """
    Translates a word or a phrase based on its structure and calculates the sentiment score.
    """
    source_lang = language_mapping.get(source_lang, source_lang)
    target_lang = language_mapping.get(target_lang, target_lang)

    cleaned_text = clean_text_for_matching(text)
    source_to_english = translation_dicts.get(source_lang, {}).get('english', {})
    english_to_target = translation_dicts['english'].get(target_lang, {})
    source_sentiments = sentiment_averages.get(source_lang, {})

    words = cleaned_text.split()
    translated_sentence = []
    total_score = 0
    i = 0

    while i < len(words):
        max_match_length = 0
        matched_phrase = None
        translated_phrase = None
        phrase_score = 0

        for j in range(i + 1, len(words) + 1):
            phrase = ' '.join(words[i:j])
            if phrase in source_to_english:
                max_match_length = j - i
                matched_phrase = phrase
                translated_phrase = english_to_target.get(source_to_english[phrase], source_to_english[phrase])
                phrase_score = source_sentiments.get(matched_phrase, 0)

        if matched_phrase:
            translated_sentence.append(str(translated_phrase) if translated_phrase else "")
            total_score += phrase_score
            i += max_match_length
        else:
            word = words[i]
            translated_word = english_to_target.get(source_to_english.get(word, word), source_to_english.get(word, word))
            word_score = source_sentiments.get(word, 0)
            translated_sentence.append(str(translated_word) if translated_word else "")
            total_score += word_score
            i += 1

    translated_text = ' '.join(translated_sentence)
    sentiment = "neutral"
    if total_score > 0.05:
        sentiment = "positive"
    elif total_score < -0.05:
        sentiment = "negative"

    return translated_text, total_score, sentiment

# Function to safely translate and analyze sentiment
def safe_translate_and_analyze_sentiment(text, source_lang, target_lang, translation_dicts, sentiment_averages):
    if isinstance(text, str):
        return translate_and_analyze_sentiment(text, source_lang, target_lang, translation_dicts, sentiment_averages)
    else:
        return "", 0, "neutral"

# Load the uploaded test corpus file
test_corpus_file = list(uploaded_test_corpus.keys())[0]
test_df = pd.read_excel(test_corpus_file)

# Apply the updated translation and sentiment function to each row in the test corpus
translated_sentences = []
total_scores = []
sentiments = []

for index, row in test_df.iterrows():
    sentence = row['sentence']
    source_lang = row['source_language'].lower()
    target_lang = row['target_language'].lower()

    translated_text, total_score, sentiment = safe_translate_and_analyze_sentiment(
        sentence, source_lang, target_lang, translation_dicts, sentiment_averages
    )
    translated_sentences.append(translated_text)
    total_scores.append(total_score)
    sentiments.append(sentiment)

# Update the DataFrame with the translated words or phrases, total scores, and sentiment
test_df['translated_text'] = translated_sentences
test_df['total_score'] = total_scores
test_df['sentiment'] = sentiments

# Display the results using pandas to show in Colab
test_df[['sentence', 'source_language', 'target_language', 'translated_text', 'total_score', 'sentiment']]


Please upload your lexicon Excel file (e.g., 'corrected_lexicon.xlsx'):


Saving corrected_lexicon.xlsx to corrected_lexicon.xlsx
Please upload your testing corpus Excel file (e.g., 'test_corpus.xlsx'):


Saving test_corpus.xlsx to test_corpus.xlsx


sentence source_language  \
0               Arrange Rub door Creator Field         english   
1            Rearrange blood Snail Fog Poverty         english   
2     Speak Engineer chicken Coordinator grief         english   
3         Speak again justice beef Boy whisper         english   
4      Put back window shot down repeated many         english   
...                                        ...             ...   
2994     she has Bear amour Savior To put back         english   
2995    We have shot down Suffering Water Find         english   
2996     you have Deceased chew vigilant Truth         english   
2997          for them oak fear Glory To write         english   
2998             And Beauty side Bible Sadness         english   

     target_language                                    translated_text  \
0             ciluba          kulongolola kulaba tshiibi mufuki bilanga   
1             french       réarranger sang escargot brouillard pauvreté   
2          afrikaans     praat ingenieur hoender koördineerder hartseer   
3               zulu  khuluma again ubulungiswa inyama yenkomo umfan...   
4             sepedi  bea  morago letsikangope shot down bušeletša b...   
...              ...                                                ...   
2994          sepedi   nan has rwala ditlhamo mophološi nan bea  morago   
2995          ciluba            beto wana shot down makenga nkasa tomba   
2996          french       tu/vous avoir décédée mâcher vigilant vérité   
2997       afrikaans                 vir them eik vrees glorie na skryf   
2998            zulu           futhi ubuhle usebe ibhayibheli ukudabuka   

      total_score sentiment  
0       14.400000  positive  
1        7.166667  positive  
2        6.666667  positive  
3       12.833333  positive  
4        8.571429  positive  
...           ...       ...  
2994    10.321429  positive  
2995    11.277273  positive  
2996     5.833333  positive  
2997     6.666667  positive  
2998     3.509524  positive  

[2999 rows x 6 columns]